In [1]:
from winq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
import mplfinance as mpf
from winq.data.stockdb import StockDB
from winq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json
from winq.analyse.tools import *
from winq.analyse.plot import *

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()

data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

2021-02-12 21:26:01,473 - StockDB - DEBUG - 加载股票日线, kwargs={'filter': {'code': 'sz002132'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2021-02-12 21:26:01,635 - StockDB - DEBUG - 加载日线数据成功 size=60
2021-02-12 21:26:01,637 - StockDB - DEBUG - 加载股票信息, kwargs={'filter': {'code': 'sz002132'}, 'projection': ['name']} ...
2021-02-12 21:26:01,714 - StockDB - DEBUG - 加载股票信息成功 size=1
2021-02-12 21:26:01,719 - FundDB - DEBUG - 加载场内基金日线数据, kwargs={'filter': {'code': '159949'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2021-02-12 21:26:01,836 - FundDB - DEBUG - 加载场内基金日线数据,成功, size=60
2021-02-12 21:26:01,837 - FundDB - DEBUG - 加载基金列表, kwargs={'filter': {'code': '159949'}, 'projection': ['short_name']}
2021-02-12 21:26:01,917 - FundDB - DEBUG - 加载基金列表成功, size=1


In [2]:
# data = data_fund

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]


def kline_tooltip_fmt_func():
    return JsCode("""function(obj){
                function tips_str(pre, now, tag, field, unit) {
                    var tips = tag + ':&nbsp;&nbsp;&nbsp;&nbsp;';
                    var span = '';
                    
                    if (field == 'open' || field == 'close' ||
                        field == 'low' || field == 'high') {
                        var fixed = 3;
                        if (now['code'].startsWith('s')) {
                            fixed = 2;
                        }
                        if (pre != null) {
                            var chg = (now[field] - pre['close']).toFixed(fixed);
                            var rate = (chg * 100 / pre['close']).toFixed(2);

                            if (rate >= 0) {
                                span = '<span style="color: #F11300;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            } else {
                                span = '<span style="color: #00A800;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            }

                        } else {
                            span = now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + rate + ')<br>';
                        }
                    } else {
                        if (field == 'volume') {
                            span = (now[field] / 1000000.0).toFixed(2) + '&nbsp;万手<br>';
                        }
                        if (field == 'turnover') {
                            span = now[field] + '%<br>'
                        }
                    }
                    return tips + span;

                }
                
                var pre_data = null;
                var now_data = kdata[trade_date[obj.dataIndex]];
                if (obj.dataIndex > 0) {
                    pre_data = kdata[trade_date[obj.dataIndex-1]];
                }
                var title = now_data['code'] + '&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '<br><br>';

                if ('name' in now_data) {
                    title = now_data['name'] + '(' + now_data['code'] + ')&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '</div><br>';
                }
                
                return title + 
                    tips_str(pre_data, now_data, '开盘', 'open') + 
                    tips_str(pre_data, now_data, '最高', 'high') + 
                    tips_str(pre_data, now_data, '最低', 'low') + 
                    tips_str(pre_data, now_data, '收盘', 'close') + 
                    tips_str(pre_data, now_data, '成交', 'volume') + 
                    tips_str(pre_data, now_data, '换手', 'turnover');
                 }""")

def kline_orig_data(data):
    origData = data[:]
    origData['trade_date'] = origData['trade_date'].apply(lambda d: d.strftime('%Y-%m-%d'))
    origData.index = origData['trade_date']
    origData.fillna('-', inplace=True)
    return origData.to_dict('index')

kline = Kline()
kline.add_js_funcs('var kdata={}'.format(kline_orig_data(data)))
kline.add_js_funcs('var trade_date={}'.format(trade_date))
kline.add_xaxis(trade_date)
kline.add_yaxis('日线', k_data,
                itemstyle_opts=opts.ItemStyleOpts(
                    color='#F11300',
                    color0='#00A800',
                ),
#                markline_opts=opts.MarkLineOpts(
#                    data=[opts.MarkLineItem(type_="max", value_dim="highest")])
                tooltip_opts=opts.TooltipOpts(
                    formatter=kline_tooltip_fmt_func())
               )
kline.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
#                        datazoom_opts=[
#                 opts.DataZoomOpts(
#                     is_show=False, type_="inside", is_zoom_lock=True, xaxis_index=[0, 0], range_end=100
#                 ),
#                 opts.DataZoomOpts(
#                     is_show=True, is_zoom_lock=True, xaxis_index=[0, 1], pos_top="95%", range_end=100
#                 ),
# #                 opts.DataZoomOpts(is_show=False, xaxis_index=[0, 2], range_end=100),
#             ],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))



ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

ma10 = MA(data['close'], timeperiod=10)
ma10 = [round(v, 3) for v in ma10]

wma = WMA(data['close'], timeperiod=2)

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('ma10', ma10, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('wma', data['close'], label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')

kline.overlap(line)

kline.render_notebook()
# bar = Bar()
# bar.add_xaxis(xaxis_data=trade_date)
# bar.add_yaxis(
#             "Volumn", list(data["volume"]/10000),
#         xaxis_index=1,
#             yaxis_index=1,
#             label_opts=opts.LabelOpts(is_show=False),
#             itemstyle_opts=opts.ItemStyleOpts(
#                 color=JsCode(
#                     """
#                 function(obj) {
#                     var now_data = kdata[trade_date[obj.dataIndex]];
#                     if (now_data['close'] > now_data['open']) {
#                         return '#F11300';
#                     }
#                     return '#00A800';
#                 }
#                 """
#                 )
#             ),
#         )

# bar.set_global_opts(
#             xaxis_opts=opts.AxisOpts(
#                 axislabel_opts=opts.LabelOpts(is_show=False),
#             ),
#             legend_opts=opts.LegendOpts(is_show=False),
#             datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],

#         )

# grid_chart = Grid()
# grid_chart.add_js_funcs('var barData={}'.format(list(data["volume"])))
# grid_chart.add_js_funcs('var kdata={}'.format(kline_orig_data(data)))
# grid_chart.add_js_funcs('var trade_date={}'.format(trade_date))

# grid_chart.add(
#         kline,
#         grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%", height="60%"),
#     )
# grid_chart.add(
#         bar,
#         grid_opts=opts.GridOpts(
#             pos_left="3%", pos_right="1%", pos_top="75%", height="20%"
#         ),
#     )
# grid_chart.render_notebook()
#

In [2]:
kline = plot_kline(data)

In [3]:
kline.render_notebook()

In [6]:
from pyecharts import options as opts
from pyecharts.charts import EffectScatter
from pyecharts.faker import Faker
from pyecharts.globals import SymbolType

In [128]:
def kline_tooltip_fmt_func():
    return JsCode(
        """
        function (obj) {
            var data = obj['data'];
            return '卖出: ' + obj['value'];
        }
        """
    )

x = Faker.choose()
y = Faker.values()
print(x)
print(y)
scatter = Scatter()
scatter.add_xaxis(x + x)
scatter.add_yaxis("", y, symbol_size=15,
                 itemstyle_opts=opts.ItemStyleOpts(color='green'))
scatter.set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))

scatter.render_notebook()

['小米', '三星', '华为', '苹果', '魅族', 'VIVO', 'OPPO']
[27, 145, 54, 88, 106, 87, 49]


In [1]:
k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]
ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

l = plot_chart(Line, trade_date, ma5,  'demo')

def kline_tooltip_fmt_func():
    return JsCode(
        """
        function (obj) {
            var data = obj['data'];
            return '卖出: ' + obj['value'];
        }
        """
    )
scatter_data = [ma5[0]*1.01, ma5[5] *1.01, ma5[10]*1.01]
trade_date = [trade_date[0], trade_date[5], trade_date[10]]

s = plot_chart(Scatter, x_index=trade_date, y_data=scatter_data,  title='',
               symbol=SymbolType.PIN,
               symbol_rotate=180
                    )
# scatter = EffectScatter()
# scatter.add_xaxis(trade_date)
# scatter.add_yaxis('scattter', scatter_data, symbol=SymbolType.ARROW, symbol_rotate=180,
#                   tooltip_opts=opts.TooltipOpts(
#                         formatter=kline_tooltip_fmt_func())
#                     )
# scatter.set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))

# scatter.render_notebook()
l = plot_overlap(l, s)

l.render_notebook()

NameError: name 'data' is not defined

In [24]:
Faker.values()

[50, 128, 116, 49, 45, 20, 119]

In [54]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line
from pyecharts.faker import Faker
import random

kdata = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y/%m/%d')[2:] for d in data['trade_date']]


def buy_fmt_func():
    print('ff')
    return JsCode(
        """
        function (obj) {
            var data = obj['data'];
            return '卖出: ' + obj['value'];
        }
        """
    )

t_data=list(data['open'])

print(trade_date[15], t_data[15])
    
chart = Line()
chart.add_xaxis(xaxis_data=trade_date)
chart.add_yaxis(series_name='line1', y_axis=list(data['open']),
                        label_opts=opts.LabelOpts(is_show=False),
                        is_smooth=False, symbol='none',
                itemstyle_opts=opts.ItemStyleOpts(color='#FD5532'),
                markpoint_opts=opts.MarkPointOpts(
                    label_opts=opts.LabelOpts(
                        formatter=buy_fmt_func
                    ),
                    data=[
                        opts.MarkPointItem(
                            symbol='path://M752.64 376.32 546.56 202.88c-19.2-16-50.56-16-69.76 0L270.72 376.32C240 402.56 261.76 447.36 305.28 447.36L384 447.36C384 447.36 384 448 384 448l0 320c0 35.2 28.8 64 64 64l128 0c35.2 0 64-28.8 64-64L640 448c0 0 0-0.64 0-0.64l78.08 0C761.6 447.36 783.36 402.56 752.64 376.32z',
                            symbol_size=10,
                            coord=[trade_date[15], t_data[15] * 0.99]
                        ),
                        opts.MarkPointItem(
                            symbol='path://M719.36 575.36l-77.44 0c0-0.64 0-0.64 0-1.28L641.92 256c0-35.2-28.8-64-64-64L448 192C412.8 192 384 220.8 384 256l0 318.08c0 0.64 0 0.64 0 1.28L305.92 575.36c-44.16 0-65.92 44.8-35.2 70.4l206.72 173.44c19.2 16 50.56 16 69.76 0l206.72-173.44C785.28 620.16 763.52 575.36 719.36 575.36z',
                            symbol_size=10,
                            coord=[trade_date[20], t_data[20] * 1.01]
                        ),
                    ]
                ),
                markline_opts=opts.MarkLineOpts(
                    symbol='none',
                    data=[
                        opts.MarkLineItem(
                            symbol='none',
                            name='高能预警',
                            x=20,
                        ),
                        opts.MarkLineItem(
                            y=3
                        ),
                    ],
                    label_opts=opts.LabelOpts(position='insideMiddle')
                ),
               )
chart.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, is_scale=True, ),
                      yaxis_opts=opts.AxisOpts(
                          position="right",
                          is_scale=True,
                          splitarea_opts=opts.SplitAreaOpts(
                              is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                          ),

                      ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      legend_opts=opts.LegendOpts(pos_left='10%', orient='vertical'),
                      graphic_opts=opts.GraphicGroup(
                          graphic_item=opts.GraphicItem(
                              left="105px",
                              top="50px",
                          ),
                          children=[
#                               opts.GraphicText控制文字的显示
                              opts.GraphicRect(
                                  graphic_item=opts.GraphicItem(
                                      z=-100,
                                      left="left",
                                      top="middle",
                                  ),
                                  graphic_shape_opts=opts.GraphicShapeOpts(
                                      width=190, height=70,
                                  ),
                                  graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                      fill="#fff", stroke="#555", line_width=1, shadow_blur=8,
                                      shadow_offset_x=3, shadow_offset_y=3, shadow_color="rgba(0,0,0,0.3)",
                                  ),
                              ),
                              # opts.GraphicText控制文字的显示
                              opts.GraphicText(
                                  graphic_item=opts.GraphicItem(left="left", top="middle",z=-100,),
                                  graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                                      # 可以通过jsCode添加js代码，也可以直接用字符串
                                      text=JsCode(
                                          "['  横轴表示数据类别',"
                                              "'纵轴表示数值的值',"
                                              "'这个文本块可以放在图中各',"
                                              "'种位置'].join('\\n')"
                                      ),
                                      font="12px bold",
                                      graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(fill='#F11300')
                                  )
                              )
                          ]
                        ),
                      )
line1 = chart

# line_t = plot_chart(Line, x_index=[trade_date[5], trade_date[5]], y_data=[list(data['close'])[5], 0],  title='k2')
# line1 = plot_overlap(line1, line_t)

line2 = plot_chart(Line, x_index=trade_date, y_data=list(data['close']),  title='k2')
line1 = plot_overlap(line1, line2)
                  
line2 = plot_chart(Line, x_index=trade_date, y_data=list(data['close']),  title='k2')

scatter_data = list(data['close'] * 1.01)
trade_date.append('21/01/01')
scatter_data.append('4.5')

s = plot_chart(Scatter, x_index=trade_date, y_data=scatter_data,  title='sc',
               symbol='path://M752.64 376.32 546.56 202.88c-19.2-16-50.56-16-69.76 0L270.72 376.32C240 402.56 261.76 447.36 305.28 447.36L384 447.36C384 447.36 384 448 384 448l0 320c0 35.2 28.8 64 64 64l128 0c35.2 0 64-28.8 64-64L640 448c0 0 0-0.64 0-0.64l78.08 0C761.6 447.36 783.36 402.56 752.64 376.32z',
               symbol_rotate=180, with_global_opts=True
                    )
kline = plot_kline(data, with_global_opts=True)
kline = plot_overlap(kline, s)

kline2 = plot_kline(data, with_global_opts=True)
kline2 = plot_overlap(kline2, s)

grid = Grid(init_opts=opts.InitOpts(height='850px', width='900px'))
    
grid.add(line1, grid_opts=opts.GridOpts(pos_top="50px", height="100px", is_show=True))
grid.add(line1, grid_opts=opts.GridOpts(pos_top="150px", height="100px", is_show=True))
grid.add(kline, grid_opts=opts.GridOpts(pos_top="250px", height="250px", is_show=True))
grid.add(kline2, grid_opts=opts.GridOpts(pos_top="500px", height="250px", is_show=True))

grid.render_notebook()



20/11/13 3.53


In [122]:
def _plot_cash(data, pos):
    x_data = list(data['trade_date'])

    y_data = list(data['open'])
    # line cash
    line_cash = Line()
    line_cash.add_xaxis(xaxis_data=x_data)
    line_cash.add_yaxis(
        series_name='资金', y_axis=y_data,
        label_opts=opts.LabelOpts(is_show=False),
        is_smooth=False, symbol='none',
        itemstyle_opts=opts.ItemStyleOpts(color='#F11300'))
    
#     line_cash.set_global_opts(
#         datazoom_opts=[opts.DataZoomOpts(pos_bottom="25%", filter_mode='none')],
#         legend_opts=opts.LegendOpts(pos_left='10%', orient='vertical'),
#         graphic_opts=opts.GraphicGroup(
#             graphic_item=opts.GraphicItem(left='30%', top='10%', ),
#             children=[
#                 opts.GraphicText(
#                     graphic_item=opts.GraphicItem(left="left", top="middle", z=-100, ),
#                     graphic_textstyle_opts=opts.GraphicTextStyleOpts(
#                         text=f'  --资金tttt({y_data[-1]})\n  --资金2({y_data[-1]})', font='12px bold',
#                         graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(fill='#F11300')
#                     )
#                 )
#             ],
#         )
#     )


    y_data = list(data['close'])
    max_y, min_y = max(y_data), min(y_data)
    max_rate = round((max_y - 10) / 10 * 100, 2),
    min_rate = round((min_y - 10) / 10 * 100, 2)
    # line net
    line_net = Line()
    line_net.add_xaxis(xaxis_data=x_data)
    line_net.add_yaxis(
        series_name='净值', y_axis=y_data,
        label_opts=opts.LabelOpts(is_show=False),
        is_smooth=False, symbol='none',
        itemstyle_opts=opts.ItemStyleOpts(color='#00A800'),
        markline_opts=opts.MarkLineOpts(symbol='none', data=[
            opts.MarkLineItem(name=f'MAX({max_y}, {max_rate}%)', y=max_y, symbol='none'),
            opts.MarkLineItem(name=f'MIN({min_y}, {min_rate}%)', y=min_y, symbol='none'),
        ],
                                        label_opts=opts.LabelOpts(position='insideMiddle')
                                        ),
        
    )

    

    main_line = plot_overlap(line_cash, line_net)
    main_line.set_global_opts(
        xaxis_opts=opts.AxisOpts(is_show=False, is_scale=True),
        yaxis_opts=opts.AxisOpts(
            position=pos,
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="25%", xaxis_index=[0, 0],
                                         is_show=False, filter_mode='none'),
                       opts.DataZoomOpts(pos_bottom="25%", xaxis_index=[0, 1],
                                         is_show=True, filter_mode='none')],
        legend_opts=opts.LegendOpts(pos_left='10%', orient='vertical'),
        graphic_opts=opts.GraphicGroup(
            graphic_item=opts.GraphicItem(left='10%', top='15%', ),
            children=[
                opts.GraphicText(
                    graphic_item=opts.GraphicItem(left="left", top="25%", z=-100, ),
                    graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                        text=f'  --资金({y_data[-1]})', font='12px bold',
                        graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(fill='#00A800')
                    )
                ),
                opts.GraphicText(
                    graphic_item=opts.GraphicItem(left="left", top="150px", z=-100, ),
                    graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                        text=f'  --资金ttttt({y_data[-1]})\n  --资金ttttt2({y_data[-1]})', font='12px bold',
                        graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(fill='#F11300')
                    )
                )
            ],
        )
    )

    return main_line

line1 = _plot_cash(data, 'left')
line2 = _plot_cash(data, 'right')
grid = Grid(init_opts=opts.InitOpts(height='450px', width='900px'))
    
grid.add(line1, grid_opts=opts.GridOpts(pos_top="50px", height="100px", is_show=True))
grid.add(line2, grid_opts=opts.GridOpts(pos_top="150px", height="100px", is_show=True))


grid.render_notebook()


In [1]:

from winq.trade.report import Report
import pandas as pd
import winq.fetch as fetch

class Account:
    cash_init = 100000
    color_up = '#F11300'
    color_down = '#00A800'
    class Trader:
        def is_backtest(self):
            return True
    trader = Trader()
    db_data = None
    


report = Report(Account())
report.acct_his = pd.read_csv('/Users/luoguochun/Downloads/acct_his.csv', index_col=0)
report.deal_his = pd.read_csv('/Users/luoguochun/Downloads/deal_his.csv', index_col=0)
report.is_ready = True
report.trade_date = []
      
for trade_date in list(pd.date_range('2020-12-01', '2020-12-31')):
    if fetch.is_trade_date(trade_date):
        report.trade_date.append(trade_date.strftime('%Y/%m/%d')[2:])


In [2]:
line = report.plot_profit()
line.render_notebook()